# Этап 1: Продвинутый анализ потенциала сжатия

Мы уже знаем размер модели. Теперь давайте выясним, насколько она «хрупкая» и где прячутся главные враги квантования.

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from src.model import GPTLanguageModel, device

# Загружаем модель (желательно уже обученную)
model = GPTLanguageModel().to(device)
try:
    model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
    print("✅ Загружены обученные веса из model_ckpt.pt")
except:
    print("⚠️ Чекпоинт не найден, используем случайные веса (результаты будут менее точными)")

model.eval()

/Users/vladmac/Code/ML-Colab/Andrey Karpathy Lessons/Lesson 2. nanoGPT/src/model.py:82: SyntaxWarning: invalid escape sequence '\s'
  Формула: $Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$


Using device: mps
✅ Загружены обученные веса из model_ckpt.pt


GPTLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=

### 1. Анализ Выбросов (Outliers Analysis)
Выбросы — это значения весов, которые сильно выбиваются из общего распределения. Они заставляют квантователь выбирать слишком широкий диапазон, из-за чего 99% обычных весов теряют точность.

In [2]:
def analyze_outliers(mdl, threshold_sigma=3.0):
    print(f"{'Layer Name':<40} | {'Max Val':<10} | {'Outliers %':<10}")
    print("-" * 70)
    
    for name, param in mdl.named_parameters():
        if 'weight' in name and param.dim() > 1:
            data = param.data.cpu().numpy().flatten()
            mean = np.mean(data)
            std = np.std(data)
            
            # Считаем сколько весов выходят за пределы N сигм
            outliers = np.sum(np.abs(data - mean) > threshold_sigma * std)
            outlier_pct = (outliers / len(data)) * 100
            
            print(f"{name[:40]:<40} | {np.max(np.abs(data)):<10.4f} | {outlier_pct:<10.2f}%")

analyze_outliers(model)

Layer Name                               | Max Val    | Outliers %
----------------------------------------------------------------------
token_embedding_table.weight             | 0.1620     | 0.37      %
position_embedding_table.weight          | 0.1945     | 0.39      %
blocks.0.sa.heads.0.key.weight           | 0.1467     | 0.45      %
blocks.0.sa.heads.0.query.weight         | 0.1271     | 0.33      %
blocks.0.sa.heads.0.value.weight         | 0.0829     | 0.26      %
blocks.0.sa.heads.1.key.weight           | 0.1676     | 0.46      %
blocks.0.sa.heads.1.query.weight         | 0.1550     | 0.32      %
blocks.0.sa.heads.1.value.weight         | 0.0869     | 0.30      %
blocks.0.sa.heads.2.key.weight           | 0.1391     | 0.45      %
blocks.0.sa.heads.2.query.weight         | 0.1426     | 0.31      %
blocks.0.sa.heads.2.value.weight         | 0.0716     | 0.27      %
blocks.0.sa.heads.3.key.weight           | 0.1502     | 0.51      %
blocks.0.sa.heads.3.query.weight         | 0.1

### 2. Saliency: Какие слои важнее?
Мы добавим к весам шум разной амплитуды и посмотрим, как это влияет на Loss. Это покажет нам «чувствительность» модели к порче данных.

In [3]:
from src.model import get_batch

def measure_sensitivity(mdl, noise_level=0.01):
    xb, yb = get_batch('val')
    
    # Базовый лосс
    with torch.no_grad():
        _, base_loss = mdl(xb, yb)
    
    print(f"Базовый Loss на валидации: {base_loss.item():.4f}\n")
    print(f"{'Layer to noise':<40} | {'Delta Loss':<10} | {'Sensitivity'}")
    print("-" * 75)

    for name, param in mdl.named_parameters():
        if 'weight' in name and param.dim() > 1:
            orig_data = param.data.clone()
            
            # Добавляем шум
            noise = torch.randn_like(param.data) * noise_level
            param.data.add_(noise)
            
            with torch.no_grad():
                _, new_loss = mdl(xb, yb)
            
            delta = new_loss.item() - base_loss.item()
            
            # Оценка: чем выше Sensitivity, тем опаснее сжимать этот слой
            sensitivity = "🔴 HIGH" if delta > 0.05 else ("🟡 MED" if delta > 0.01 else "🟢 LOW")
            
            print(f"{name[:40]:<40} | {delta:<10.4f} | {sensitivity}")
            
            # Возвращаем веса назад!
            param.data.copy_(orig_data)

measure_sensitivity(model)

Базовый Loss на валидации: 1.5167

Layer to noise                           | Delta Loss | Sensitivity
---------------------------------------------------------------------------
token_embedding_table.weight             | 0.0144     | 🟡 MED
position_embedding_table.weight          | 0.0127     | 🟡 MED
blocks.0.sa.heads.0.key.weight           | -0.0002    | 🟢 LOW
blocks.0.sa.heads.0.query.weight         | 0.0005     | 🟢 LOW
blocks.0.sa.heads.0.value.weight         | 0.0003     | 🟢 LOW
blocks.0.sa.heads.1.key.weight           | 0.0000     | 🟢 LOW
blocks.0.sa.heads.1.query.weight         | -0.0000    | 🟢 LOW
blocks.0.sa.heads.1.value.weight         | 0.0018     | 🟢 LOW
blocks.0.sa.heads.2.key.weight           | 0.0003     | 🟢 LOW
blocks.0.sa.heads.2.query.weight         | -0.0005    | 🟢 LOW
blocks.0.sa.heads.2.value.weight         | 0.0002     | 🟢 LOW
blocks.0.sa.heads.3.key.weight           | -0.0000    | 🟢 LOW
blocks.0.sa.heads.3.query.weight         | -0.0001    | 🟢 LOW
blocks.0.sa.hea

### Выводы Этапа 1:
1. **Outliers**: Если процент выбросов высок, обычное квантование (Min-Max) сильно уронит точность. Нам понадобятся методы вроде **Clamping** или **SmoothQuant**.
2. **Sensitivity**: Слои с пометкой 🔴 HIGH нужно сохранять в максимальной точности (например, оставить в FP16) или использовать для них большее количество бит.